# This script contains the following:
## 1. Import libraries and data
## 2. Data preprocessing
## 3. Merge coordinates to new df
## 4. Plot on map

### 1. Import libraries and data

In [1]:
# Import libraries
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

/opt/anaconda3/envs/citi-bike-py10/lib/python3.10/site-packages/keplergl/keplergl.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [2]:
# Import data
df = pd.read_csv("citibike_weather_merged_2022.csv", low_memory=False)


In [3]:
# Check import
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avg_temp
0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual,2022-08-27,27.8
1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual,2022-08-20,27.9
2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member,2022-08-31,25.6
3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.10,40.744219,-73.971212,40.764397,-73.973715,member,2022-08-02,26.4
4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member,2022-08-25,28.1


In [4]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avg_temp'],
      dtype='object')

In [5]:
# Ensure types
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [6]:
# Check for missing values
df.isna().sum().sort_values(ascending=False)


end_station_name      69884
end_station_id        69884
end_lat               37223
end_lng               37223
start_station_name       49
start_station_id         49
ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_lat                 0
start_lng                 0
member_casual             0
date                      0
avg_temp                  0
dtype: int64

I have 69,884 missing from end_station. These rides ended outside a named station (for example, the user may have parked in an area without a designated dock).

37,223 misisng coordinates. These don’t have GPS coordinates for the end location — cannot be mapped.

There are 49 missing start stations, this is small so could be dropped.

Because I need coordinates to map, I need to drop these misisng values in the coordinate columns


In [7]:
# Create a cleaned df with dopped missing rows 
df_clean = df.dropna(subset=['start_lat', 'start_lng', 'end_lat', 'end_lng']).copy()


In [8]:
# Check
df_clean.isna().sum().sort_values(ascending=False)


end_station_name      32661
end_station_id        32661
start_station_name       49
start_station_id         49
ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_lat                 0
start_lng                 0
end_lat                   0
end_lng                   0
member_casual             0
date                      0
avg_temp                  0
dtype: int64

### 2. Data preprocessing

In [9]:
# Create a 'trip = 1' column
df_clean['trip'] = 1

In [10]:
# Aggregate
df_grouped = (
    df_clean
    .groupby(['start_station_name', 'end_station_name'], as_index=False)['trip']
    .sum()
    .rename(columns={'trip': 'trip_count'})
)

In [11]:
# Check dataframe
df_grouped.head()

,start_station_name,end_station_name,trip_count
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12


In [12]:
print(df_grouped['trip_count'].sum())
print(df.shape)
print (df_clean.shape)

29768282
(29838166, 15)
(29800943, 16)


In [13]:
# Check for any missing coordinate values
df_clean[['start_lat', 'start_lng', 'end_lat', 'end_lng']].isna().sum()


start_lat    0
start_lng    0
end_lat      0
end_lng      0
dtype: int64

I verified the total number of trips in the grouped dataframe by summing the trip_count column. The total (2,976,282) is slightly lower than the total number of rows in the cleaned dataset (2,980,943).
This small difference is expected because some trips lacked valid start or end station names after dropping rows with missing coordinate values. These trips were excluded from the aggregation, but the overall count still represents nearly all valid trips available for mapping

Now in order to create the map, I need to add the coordinates. 

In [14]:
# Export and save
df_clean.to_csv('citi_bike_clean_2022.csv', index=False)
            

### 3. merge coordinates to new df

Python kept crashing while I tried to merge the coordinates with df_grouped so trying a different approach

In [15]:
# Ensure clean names
for col in ['start_station_name','end_station_name']:
    df_clean[col] = df_clean[col].astype(str).str.strip()

In [16]:
# Build look up tables
start_lookup = (
    df_clean[['start_station_name','start_lat','start_lng']]
    .dropna(subset=['start_station_name'])
    .drop_duplicates(subset=['start_station_name'])
    .set_index('start_station_name')
)

end_lookup = (
    df_clean[['end_station_name','end_lat','end_lng']]
    .dropna(subset=['end_station_name'])
    .drop_duplicates(subset=['end_station_name'])
    .set_index('end_station_name')
)


In [17]:
df_geo = df_grouped.copy()

In [18]:
df_geo['start_lat'] = df_geo['start_station_name'].map(start_lookup['start_lat'])
df_geo['start_lng'] = df_geo['start_station_name'].map(start_lookup['start_lng'])
df_geo['end_lat']   = df_geo['end_station_name'].map(end_lookup['end_lat'])
df_geo['end_lng']   = df_geo['end_station_name'].map(end_lookup['end_lng'])

In [19]:
# If any coords couldn’t be found, drop those pairs
df_geo = df_geo.dropna(subset=['start_lat','start_lng','end_lat','end_lng']).reset_index(drop=True)

In [20]:
print(df_geo.shape)

(1013397, 7)


In [21]:
df_geo.head()

,start_station_name,end_station_name,trip_count,start_lat,start_lng,end_lat,end_lng
0,1 Ave & E 110 St,1 Ave & E 110 St,791,40.792337,-73.93824,40.792327,-73.938300
1,1 Ave & E 110 St,1 Ave & E 18 St,2,40.792337,-73.93824,40.733812,-73.980544
2,1 Ave & E 110 St,1 Ave & E 30 St,4,40.792337,-73.93824,40.741444,-73.975361
3,1 Ave & E 110 St,1 Ave & E 39 St,1,40.792337,-73.93824,40.747140,-73.971130
4,1 Ave & E 110 St,1 Ave & E 44 St,12,40.792337,-73.93824,40.750020,-73.969053


To prepare my aggregated trip data for mapping, I created two small lookup tables containing each station’s latitude and longitude. I then mapped these coordinates onto my grouped dataset by matching station names. This lightweight approach replaces a memory-heavy merge and ensures that each start and end station pair has valid coordinates for visualisation in Kepler.gl.

In [23]:
# Export df_geo
# Export the cleaned and aggregated data for mapping
df_geo.to_csv('citibike_trips_geodata_2022.csv', index=False)

print("File exported successfully: citibike_trips_geodata_2022.csv")


File exported successfully: citibike_trips_geodata_2022.csv


In [24]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013397 entries, 0 to 1013396
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   start_station_name  1013397 non-null  object 
 1   end_station_name    1013397 non-null  object 
 2   trip_count          1013397 non-null  int64  
 3   start_lat           1013397 non-null  float64
 4   start_lng           1013397 non-null  float64
 5   end_lat             1013397 non-null  float64
 6   end_lng             1013397 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 54.1+ MB


In [25]:
df_geo.head()

,start_station_name,end_station_name,trip_count,start_lat,start_lng,end_lat,end_lng
0,1 Ave & E 110 St,1 Ave & E 110 St,791,40.792337,-73.93824,40.792327,-73.938300
1,1 Ave & E 110 St,1 Ave & E 18 St,2,40.792337,-73.93824,40.733812,-73.980544
2,1 Ave & E 110 St,1 Ave & E 30 St,4,40.792337,-73.93824,40.741444,-73.975361
3,1 Ave & E 110 St,1 Ave & E 39 St,1,40.792337,-73.93824,40.747140,-73.971130
4,1 Ave & E 110 St,1 Ave & E 44 St,12,40.792337,-73.93824,40.750020,-73.969053


### 4. Plot on map

In [26]:
# Create KeplerGL instance
m = KeplerGl(height=700, data={"citibike_trips": df_geo})


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [27]:
m

KeplerGl(data={'citibike_trips':             start_station_name       end_station_name  trip_count  start_lat …

I can't open it within Jupyter so will create map online

In [28]:
# Save an empty map config 
m.save_to_html(file_name="citibike_map_template.html")

Map saved to citibike_map_template.html!


#### Interpretation of map once filtered

After creating the initial arc map of all trips, I applied a filter to display only the routes with a trip count above 1,000. This focuses on the most common journeys between stations. 

Filtering the data this way reveals clear clusters of movement concentrated along Manhattan’s central spine, particularly between Midtown and Downtown. These areas represent high-frequency usage zones, likely linked to major transit hubs, business districts, and tourist destinations.

This filtered view makes it easier to visually interpret the city’s busiest cycling corridors and complements the overall analysis by highlighting where Citi Bike infrastructure is most heavily utilised.


#### Step 7: Exporting the Interactive Map

The final Kepler.gl map was refined and styled using the online Kepler interface.  
I exported the completed map as an HTML file (`kepler.gl.html`) without embedded data to keep the file size smaller.  
The map can be opened directly in any web browser to explore trip patterns interactively.


In [29]:
# Save Kepler map configuration
config = m.config
with open('kepler_config.json', 'w') as f:
    f.write(str(config))

print("Map configuration saved as kepler_config.json")


Map configuration saved as kepler_config.json


#### Step 8: Save Kepler.gl Configuration

Although the final version of the map was designed using the Kepler.gl web interface, 
this code cell demonstrates how the configuration would be saved programmatically 
if the map had been built directly in Jupyter. The configuration file stores the 
map layers, colour schemes, and filters for reproducibility.